# Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

import plotly.express as px

# 2. Datasets analysis

# 2.1 Data reading and pre-processing
########################################################################################################

1. In table "Players" birth_state and birth_city have Nan values. Information about birth can be found in table "player_data"
2. In table "player_data" birth day and year can be splitted in order to get information about birth year only
3. Unnamed categories can be named
4. We can see that in table "player_data" more unique names of the players than in other two tables
5. Additional features that could be calculated: playing range,
6. column "Age" would have small impact on analysis as it correlates with birth of the year
7. Columns '3P%','blank2','blanl','3PAr','GS','TOV%','USG%','3P','3PA','TOV' are not informative because they do not provide information about more then 20% of the players (approximately). This can have negative impact on the overal analysis and therefore need to be removed
8. Some tables containe duplacate names of the players. This need to be taken in attantion during analysis
9. Some names has * in their names. This need to be cleaned

In [2]:
players = pd.read_csv('/kaggle/input/dataset-homework-lesson4/Players.csv')
seasons_stats = pd.read_csv('/kaggle/input/dataset-homework-lesson4/Seasons_Stats.csv')
player_data = pd.read_csv('/kaggle/input/dataset-homework-lesson4/player_data.csv')

In [3]:
print('Length of the dataset: ',len(players),'\n')
players[:4]

Length of the dataset:  3922 



,Unnamed: 0,Player,height,weight,collage,born,birth_city,birth_state
0,0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN


In [4]:
print('Length of the dataset: ',len(seasons_stats),'\n')
seasons_stats[:4]

Length of the dataset:  24691 



,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0


In [5]:
print('Length of the dataset: ',len(player_data),'\n')
player_data[:4]

Length of the dataset:  4550 



,name,year_start,year_end,position,height,weight,birth_date,college
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke University
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State University
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,"April 16, 1947","University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",Louisiana State University


rename column "name" on "Player"

In [6]:
#seasons_stats.dtypes
#players.dtypes
player_data.dtypes

name           object
year_start      int64
year_end        int64
position       object
height         object
weight        float64
birth_date     object
college        object
dtype: object

In [7]:
player_data = player_data.fillna(0)

player_data['height'] = player_data['height'].str.replace('-','')
player_data['height']=pd.to_numeric(player_data['height'])
player_data['height'].unique()


array([610.,  69.,  72.,  61.,  66.,  67.,  63.,  65.,  60., 611.,  64.,
       510.,  70.,  62., 511.,  71.,  68.,  75.,  53.,  77.,  55.,  76.,
        73.,  59.,  58.,  57.,  74.,  56.,  nan])

- in "height" birth_date can be converted into integer value


In [8]:
player_data = player_data.rename(columns = {'name':'Player'})

In [9]:
print('Unique names in table "Player": ', len(players['Player'].unique()))
print('Unique names in table "player_data": ', len(player_data['Player'].unique()))
print('Unique names in table "seasons_stats": ', len(seasons_stats['Player'].unique()))

Unique names in table "Player":  3922
Unique names in table "player_data":  4500
Unique names in table "seasons_stats":  3922


remove * in names

In [10]:
players['Player']=players['Player'].str.replace('*','',regex=False)
seasons_stats['Player']=seasons_stats['Player'].str.replace('*','',regex=False)
player_data['Player']=player_data['Player'].str.replace('*','',regex=False)

In [11]:
print('"players" NaNs: \n', players.isnull().sum(),'\n')
print('"player_data" NaNs: \n', player_data.isnull().sum(),'\n')
print('"seasons_stats" NaNs: \n', seasons_stats.isnull().sum())

"players" NaNs: 
 Unnamed: 0       0
Player           1
height           1
weight           1
collage        349
born             1
birth_city     470
birth_state    483
dtype: int64 

"player_data" NaNs: 
 Player        0
year_start    0
year_end      0
position      0
height        1
weight        0
birth_date    0
college       0
dtype: int64 

"seasons_stats" NaNs: 
 Unnamed: 0        0
Year             67
Player           67
Pos              67
Age              75
Tm               67
G                67
GS             6458
MP              553
PER             590
TS%             153
3PAr           5852
FTr             166
ORB%           3899
DRB%           3899
TRB%           3120
AST%           2136
STL%           3899
BLK%           3899
TOV%           5109
USG%           5051
blanl         24691
OWS             106
DWS             106
WS              106
WS/48           590
blank2        24691
OBPM           3894
DBPM           3894
BPM            3894
VORP           3894
FG    

It can be seen that '3P%','blank2','blanl','3PAr','GS','TOV%','USG%','3P','3PA','TOV' contains a lot of missing information that will have an impact on the analysis accuracy. We will remove columns that contains >5000 Nan in table "seasons_stats"

In [12]:
seasons_stats = seasons_stats.drop(['3P%','blank2','blanl','3PAr','GS','TOV%','USG%','3P','3PA','TOV'], axis=1)
seasons_stats.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,MP,PER,TS%,...,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,PF,PTS
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,0.368,...,241.0,0.705,NaN,NaN,NaN,176.0,NaN,NaN,217.0,458.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,0.435,...,106.0,0.708,NaN,NaN,NaN,109.0,NaN,NaN,99.0,279.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,0.394,...,129.0,0.698,NaN,NaN,NaN,140.0,NaN,NaN,192.0,438.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,0.312,...,34.0,0.559,NaN,NaN,NaN,20.0,NaN,NaN,29.0,63.0
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,0.308,...,31.0,0.548,NaN,NaN,NaN,20.0,NaN,NaN,27.0,59.0


remove unimportant for statistical analysis information

In [13]:
seasons_stats = seasons_stats.drop(['Unnamed: 0'], axis=1)
players = players.drop(['Unnamed: 0'], axis=1)

In [14]:
print('In "players" unique name / all names: ', len(players['Player'].unique()),'/',len(players))
print('In "player_data" unique name / all names: ', len(player_data['Player'].unique()),'/',len(player_data))
print('In "seasons_stats" unique name / all names: ', len(seasons_stats['Player'].unique()),'/',len(player_data))

In "players" unique name / all names:  3920 / 3922
In "player_data" unique name / all names:  4500 / 4550
In "seasons_stats" unique name / all names:  3920 / 4550


players name is important information. Two tables provide information for some players more then once. This need to ba taken in attantion

marge all tables using left-merging option

In [15]:
df = seasons_stats.merge(players,how = 'left', on = 'Player')
df = df.drop(['Unnamed: 0_x','Unnamed: 0_y'], axis=1)
print(len(df))
df.head()

KeyError: "['Unnamed: 0_x', 'Unnamed: 0_y'] not found in axis"

In [ ]:
df.groupby(['Player','Year']).get_group(("Ed Bartels",1950))

we can see that for some players contain multiple information about games. Thus, for Ed Bartels TOT ('total') includes DNN and NYK. We want to remove TOT information

In [ ]:
def single_row(df_tem):
    if df_tem.shape[0]==1:
        return df_tem
    else:
        row=df_tem[df_tem['Tm']=='TOT']
        row['Tm']=df_tem.iloc[-1,:]['Tm']
        
df=df.groupby(['Player','Year']).apply(single_row)

In [ ]:
df.head(20)

we need to drop two columns on the left

In [ ]:
df.index=df.index.droplevel()
df.index=df.index.droplevel()

In [ ]:
df.head(20)

combine two tables df and player_data

In [ ]:
df2=df.merge(player_data,how="outer",on=["Player"])

In [ ]:
df2 = df2.fillna(0)
df2.head(20)

- drop column "birth_date" as it do not contain important information and repeats column "born"

In [ ]:
df2 = df2.drop(['birth_date'], axis=1)

# 2.2 Data Visualisation

In [ ]:
fig = px.histogram(df, x='Age', marginal='box', color_discrete_sequence=['green'], nbins=47, title='Age distribution')
fig.update_layout(bargap=0.1)
fig.show()

In [ ]:
fig = px.histogram(df, x='Year', marginal='box', color_discrete_sequence=['blue'], y='PF', nbins=47, title='Personal Fouls per year ')
fig.update_layout(bargap=0.1)
fig.show()